# Pretrained GluonCV FCN ResNet50 model optimizing using Neo

## Introduction

This is an example of GluonCV FCN ResNet50 model compilation using Neo runtime. In this demo, we will demonstrate how to compile and to host a segmentation model.

## Setup

To train the ssd mobilenet model on Amazon SageMaker, we need to setup and authenticate the use of AWS services. To start, we need an AWS account role with SageMaker access. This role is used to give SageMaker access to your data in S3.

In [ ]:
import boto3
import sagemaker
import time
import json
from sagemaker.utils import name_from_base
from sagemaker import get_execution_role 
from sagemaker.model import NEO_IMAGE_ACCOUNT
from sagemaker.fw_utils import create_image_uri

In [ ]:
role = get_execution_role() 
sess = sagemaker.Session()
region = sess.boto_region_name
account = sess.boto_session.client('sts').get_caller_identity()['Account']
framework = 'mxnet'
framework_version = '1.4.1'
bucket = sess.default_bucket()
prefix = 'segmentation'

In [ ]:
target_device = 'ml_p3'
image_url = create_image_uri(region, 
                             'neo-' + framework.lower(), 
                             target_device.replace('_', '.'),
                             framework_version, 
                             py_version='py3', 
                             account=NEO_IMAGE_ACCOUNT[region]
                            )

## Download Pretrained model using the `get_model.py` script

In [ ]:
!python3 get_model.py

### Zip the pretrained model binaries to a tarball file.

In [ ]:
!tar -zcvf fcn_resnet50_ade.tar.gz fcn_resnet50_ade-0000.params fcn_resnet50_ade-symbol.json

### Upload model to S3

In [ ]:
model_path = sess.upload_data(path='fcn_resnet50_ade.tar.gz', key_prefix=prefix)

### Load model using `sagemaker.MXNetModel`

In [ ]:
from sagemaker.mxnet.model import MXNetModel
from sagemaker.predictor import RealTimePredictor

segmentation_model = MXNetModel(model_data=model_path,
                                image=image_url,
                                entry_point='entry_point.py',
                                predictor_cls=RealTimePredictor,
                                role=role,
                                sagemaker_session=sess,
                                py_version='py3',
                                framework_version=framework_version)

### Compile model for `ml_p3` target using Neo. 

After compilation, we will deploy Neo optimized model to do inderence. 

In [ ]:

compilation_job_name = name_from_base('Segmentation-fcn-resnet50-Neo')
output_path = 's3://{}/{}'.format(bucket, prefix)
compiled_model = segmentation_model.compile(target_instance_family='ml_p3', 
                                            input_shape={'data':[1,3,480,480]},
                                            job_name=compilation_job_name,
                                            role=role,
                                            framework=framework,
                                            output_path=output_path)

## Inference

Now that the model is deployed at an endpoint that is up-and-running, we can use this endpoint for inference.

In [ ]:
object_detector = compiled_model.deploy(initial_instance_count = 1,
                                        instance_type = 'ml.p3.2xlarge')

In [ ]:
# download the example image
!wget -O test.jpg https://github.com/zhanghang1989/image-data/blob/master/encoding/segmentation/ade20k/ADE_val_00001755.jpg?raw=true

file_name = "test.jpg"

with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)

In [ ]:
%%time
object_detector.content_type = 'image/jpeg'
response = object_detector.predict(payload)

In [ ]:
import json
detections = json.loads(response)

We have provided additional script to easily visualize the detection outputs. 

In [ ]:
adepallete = [
    0,0,0,120,120,120,180,120,120,6,230,230,80,50,50,4,200,3,120,120,80,140,140,140,204,
    5,255,230,230,230,4,250,7,224,5,255,235,255,7,150,5,61,120,120,70,8,255,51,255,6,82,
    143,255,140,204,255,4,255,51,7,204,70,3,0,102,200,61,230,250,255,6,51,11,102,255,255,
    7,71,255,9,224,9,7,230,220,220,220,255,9,92,112,9,255,8,255,214,7,255,224,255,184,6,
    10,255,71,255,41,10,7,255,255,224,255,8,102,8,255,255,61,6,255,194,7,255,122,8,0,255,
    20,255,8,41,255,5,153,6,51,255,235,12,255,160,150,20,0,163,255,140,140,140,250,10,15,
    20,255,0,31,255,0,255,31,0,255,224,0,153,255,0,0,0,255,255,71,0,0,235,255,0,173,255,
    31,0,255,11,200,200,255,82,0,0,255,245,0,61,255,0,255,112,0,255,133,255,0,0,255,163,
    0,255,102,0,194,255,0,0,143,255,51,255,0,0,82,255,0,255,41,0,255,173,10,0,255,173,255,
    0,0,255,153,255,92,0,255,0,255,255,0,245,255,0,102,255,173,0,255,0,20,255,184,184,0,
    31,255,0,255,61,0,71,255,255,0,204,0,255,194,0,255,82,0,10,255,0,112,255,51,0,255,0,
    194,255,0,122,255,0,255,163,255,153,0,0,255,10,255,112,0,143,255,0,82,0,255,163,255,
    0,255,235,0,8,184,170,133,0,255,0,255,92,184,0,255,255,0,31,0,184,255,0,214,255,255,
    0,112,92,255,0,0,224,255,112,224,255,70,184,160,163,0,255,153,0,255,71,255,0,255,0,
    163,255,204,0,255,0,143,0,255,235,133,255,0,255,0,235,245,0,255,255,0,122,255,245,0,
    10,190,212,214,255,0,0,204,255,20,0,255,255,255,0,0,153,255,0,41,255,0,255,204,41,0,
    255,41,255,0,173,0,255,0,245,255,71,0,255,122,0,255,0,255,184,0,92,255,184,255,0,0,
    133,255,255,214,0,25,194,194,102,255,0,92,0,255]

In [ ]:
import PIL.Image
import numpy as np
detections = np.array(detections) + 1
out_img = PIL.Image.fromarray(detections.astype('uint8'))
out_img.putpalette(adepallete)
out_img.save('out.png')

In [ ]:
PIL.Image.open('out.png')

## Delete the Endpoint
Having an endpoint running will incur some costs. Therefore as a clean-up job, we should delete the endpoint.

In [ ]:
sess.delete_endpoint(object_detector.endpoint)